In [ ]:
import cv2
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import SimpleITK as sitk

import zipfile
from read_roi import read_roi_zip 

from  scipy import ndimage

from skimage.io import imread, imshow
from skimage import *
import skimage

import pandas as pd
from PIL import *

###########################
def win_scale(data, wl, ww, dtype, out_range):
    """
    Scale pixel intensity data using specified window level, width, and intensity range.
    """
    
    data_new = np.empty(data.shape, dtype=np.double)
    data_new.fill(out_range[1]-1)
    
    data_new[data <= (wl-ww/2.0)] = out_range[0]
    
    data_new[(data>(wl-ww/2.0))&(data<=(wl+ww/2.0))] = ((data[(data>(wl-ww/2.0))&(data<=(wl+ww/2.0))]-(wl-0.5))/(ww-1.0)+0.5)*(out_range[1]-out_range[0])+out_range[0]
    
    data_new[data > (wl+ww/2.0)] = out_range[1]-1
    
    return data_new.astype(dtype)

In [ ]:
pwd

In [ ]:
xlsx = pd.read_excel('../../local/data/dvt_list.xlsx')

In [ ]:
p_numpath= sorted(glob.glob('../../local/data/dvt/*'))
print(p_numpath[0])

In [ ]:
p_numStr=[]
p_numInt=[]
for pn in range(len(p_numpath)):
    p_numStr.append(p_numpath[pn][p_numpath[pn].rindex('/')+1:])
    p_numInt.append(int(p_numpath[pn][p_numpath[pn].rindex('/')+1:]))
print(p_numStr[0])
print(p_numInt[0])
print(len(p_numpath),len(p_numStr),len(p_numInt))

In [ ]:
print(xlsx['ID'][0])
print(xlsx)

In [ ]:
p_ids = []
ww_list = []
wl_list = []
for ids in range(len(xlsx)):
    p_ids.append(str(xlsx['ID'][ids]))
    ww_list.append(xlsx['WW'][ids])
    wl_list.append(xlsx['WL'][ids])
print(len(p_ids), len(ww_list), len(wl_list))    

In [ ]:
print(type(p_ids[30]))
print(p_ids)

In [ ]:
print(p_ids.index(p_numStr[45]))
print(p_ids[p_ids.index(p_numStr[45])], p_numStr[45])
ww=[]
wl=[]
for tt in range(len(p_numStr)):
    if p_numStr[tt] not in p_ids:
        print(p_numStr[tt], 'files not in list')
    else:
        print('number:', tt, 'match:', p_ids[p_ids.index(p_numStr[tt])], p_numStr[tt])
        if p_ids[p_ids.index(p_numStr[tt])] != p_numStr[tt]:
            print('No matching')
        else:
            ww.append(ww_list[p_ids.index(p_numStr[tt])])
            wl.append(wl_list[p_ids.index(p_numStr[tt])])

In [ ]:
print(len(p_numpath), len(ww), len(wl))
dcmfiles= glob.glob(p_numpath[0]+'/*.dcm')
print(dcmfiles[0], dcmfiles[0][dcmfiles[0].rindex('/') + 1 :dcmfiles[0].rindex('.dcm')])

In [ ]:
imgpath='../../defaultImg'

In [ ]:
for df in range(len(p_numpath)):
    os.mkdir(imgpath+ '/' + p_numStr[df])
    png_path= imgpath+ '/' + p_numStr[df]
    dcmfiles= glob.glob(p_numpath[df]+'/*.dcm')
    # print(dcmfiles)
    for dwh in range(len(dcmfiles)):

        dcmname= dcmfiles[dwh][dcmfiles[0].rindex('/') + 1 :dcmfiles[0].rindex('.dcm')]

        dcm_dcm = sitk.ReadImage(dcmfiles[dwh])
        dcm_img = sitk.GetArrayFromImage(dcm_dcm)[0]

        dcm_scaled = win_scale(dcm_img, wl[df], ww[df], np.uint8, (0, 255))

#         plt.figure(figsize=(10,10))
#         plt.imshow(dcm_scaled, cmap='gray')
        
        io.imsave(png_path+'/'+ dcmname+'.png', dcm_scaled)
